<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example" data-toc-modified-id="Example-1">Example</a></span></li><li><span><a href="#Helper-methods" data-toc-modified-id="Helper-methods-2">Helper methods</a></span></li><li><span><a href="#Mine" data-toc-modified-id="Mine-3">Mine</a></span></li><li><span><a href="#restructure-plot" data-toc-modified-id="restructure-plot-4">restructure plot</a></span></li><li><span><a href="#Processed-data" data-toc-modified-id="Processed-data-5">Processed data</a></span></li></ul></div>

In [1]:
from bokeh.io import *
from bokeh.layouts import *
from bokeh.plotting import *
from bokeh.models.renderers import *
from bokeh.palettes import *
from bokeh.models.widgets import *
from bokeh.models import *
from bokeh import events

from NearFieldOptics.Materials import *
from NearFieldOptics import TipModels as T
import copy
import numpy as np
from matplotlib import pyplot as plt

output_notebook()
# output_file("new GUI layout.html")

<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Bi2Se3_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "PMMA_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "sio2_300nm_extracted_epsilon_cone_A=2a.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_230K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_30K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Erik_BSTS_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Insulating.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Metallic.pickle"...


Loading BokehJS ...

In [2]:
from NearFieldOptics.Materials import *
from NearFieldOptics.Materials.material_types import *
from TransferMatrixMethod import MatrixBuilder,Calculator
import sympy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import scipy.special as ss
import mpmath as mp
from mpmath.ctx_fp import FPContext
from sys import stdout
sympy.init_printing(use_unicode=True)

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)
error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)
error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): NameError("name 'table' is not defined",)


True
[(<NearFieldOptics.Materials.material_types.AnisotropicMaterial object at 0x1134a3908>, 3e-06), <NearFieldOptics.Materials.material_types.SingleLayerGraphene object at 0x1a1e1b26a0>, (<NearFieldOptics.Materials.material_types.AnisotropicMaterial object at 0x1134a3908>, 3e-06)]


# Example

In [3]:
def modify_doc(doc):

    start_button = Button(label="Start the experiment") 
    fig1 = figure()
    fig1.circle([0,1,2],[1,3,2])
    fig2 = figure()
    fig2.circle([0,0,2],[4,-1,1])
    fig3 = figure()
    fig3.circle([0,4,3],[1,2,0])

    l1 = layout([[fig1, fig2]], sizing_mode='fixed')
    l2 = layout([[fig3,start_button]],sizing_mode='fixed')

    tab1 = Panel(child=l1,title="This is Tab 1")
    tab2 = Panel(child=l2,title="This is Tab 2")
    tabs = Tabs(tabs=[ tab1, tab2 ])

    doc.add_root(tabs)

show(modify_doc)

# Helper methods

In [4]:
def header():
    text = Div(text="<h1><b>The Lightning Rod Model GUI</b></h1>",width=400, height=30)
    title = WidgetBox(text, width=400)

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    cu = figure(x_range=(0,6), y_range=(-3.5,0.2),width=250,height=60,toolbar_location=None, tools=[tap])
    cu.outline_line_color = "white"
    cu.image_url(url=['Columbia logo.png'], x=0, y=0.2,w=6, h=3.5)
    cu.xgrid.grid_line_color = None
    cu.ygrid.grid_line_color = None
    cu.axis.visible = False

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    basov = figure(x_range=(0,6), y_range=(-3.5,0.2),width=150,height=60,toolbar_location=None, tools=[tap])
    basov.outline_line_color = "white"
    basov.image_url(url=['basov lab logo.png'], x=0, y=0.2,w=6, h=3.5)
    basov.xgrid.grid_line_color = None
    basov.ygrid.grid_line_color = None
    basov.axis.visible = False
    return [title,cu,basov]

def space():
    empty = Div(text="",width=400, height=1)
    space = WidgetBox(empty, width=400)
    return [empty]

def _get_index_list_(layerList):
    numList = list(range(1,len(layerList)+1,1))
    indexList = list()
    for i in numList:
        indexList.append(str(i))
    return indexList

def _get_computer_material_(matString):
        
        if matString == "SiO2":
            return SiO2_300nm
        elif matString == "BN_Caldwell":
            return BN_Caldwell
        elif matString == "FePO4":
            return FePO4
        elif matString == "TaS2_CCDW":
            return TaS2_CCDW
        elif matString == "TaS2_NCCDW":
            return TaS2_NCCDW
        elif matString == "Silicon":
            return Si
        elif matString == "Air":
            return Air
        elif matString == "Au":
            return Au
        else:
            Logger.raiseException('Material not recognized.', exception=ValueError)
            return None

def _get_name_of_LM(layerList,computerLayerList):
    string = ''
    for i in range(0,len(layerList)-1,1):
        if type(computerLayerList[i])==SingleLayerGraphene:
            string = string+layerList[i][0]+' - '
        else:
            string = string+layerList[i][0] + '(' + str(int(layerList[i][1])) + 'nm)' + ' - '

    if type(computerLayerList[len(layerList)-1])==SingleLayerGraphene:
        string = string+layerList[len(layerList)-1][0]
    else:
        string = string+layerList[len(layerList)-1][0] + '(' + str(int(layerList[len(layerList)-1][1])) + 'nm)'
    return string

# Mine

In [5]:
# def modify_doc(doc):
    
#     #################################
#     ##### tab 1: Material input #####
#     ##################################################################################################################
#     global layerList,computerLayerList,tabulatedData
#     tabulatedData = {}
#     layerList = []
#     computerLayerList = []
    
#     material_list = ["Au","SiO2","SingleLayerGraphene","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
#     material_list_without_surface = copy.copy(material_list)
#     material_list_without_surface.remove("SingleLayerGraphene")
    
#     entranceMaterialMenu = Select(title="Entrance material:", value="Air", options=sorted(material_list))
#     exitMaterialMenu = Select(title="Exit material:", value="Air", options=sorted(material_list))
#     materialMenu = Select(title="Material", options=sorted(material_list), value="SiO2",width=200)
    
#     thicknessText = TextInput(title="Thickness (nm):", value="30")
#     muText = TextInput(title="Chemical potential (cm-1):", value="1000")
#     gammaText = TextInput(title="Scattering rate ():", value="0.1")
    
#     add = Button(label="Add", button_type="primary")
#     assemble = Button(label="ASSEMBLE!", button_type="success")
#     clear = Button(label="CLEAR ALL", button_type="warning")
    
#     tabulatedData["material"] = []
#     tabulatedData["thickness"] = []
#     tabulatedData["mu"] = []
#     tabulatedData["gamma"] = []
    
#     source1 = ColumnDataSource(tabulatedData)
#     source = source1
#     columns = [
#             TableColumn(field="material", title="Material"),
#             TableColumn(field="thickness", title="thickness (nm)"),
#             TableColumn(field="mu", title="chemical potential (cm-1)"),
#             TableColumn(field="gamma", title="scatting rate ()"),
#         ]
#     table = DataTable(source=source, columns=columns, width=700, height=280)
    
#     select_code = """
#     to_show = cb_obj.value;
#     options = cb_obj.options

#     for (i=0;i<options.length;i++) {
#         box = document.getElementsByClassName(options[i])[0];
#         if (!box.className.includes("hidden")) {box.className+=" hidden";}
#         if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
#     }
#     """
    
#     materialMenu.callback = CustomJS(code=select_code)
    
#     #Add a layer
#     def update_material_parameters():
        
#         if materialMenu.value=="SingleLayerGraphene":
#             layerList.append(("SingleLayerGraphene",muText.value,gammaText.value))
#             tabulatedData["material"].append("SingleLayerGraphene")
#             tabulatedData["thickness"].append("/")
#             tabulatedData["mu"].append(muText.value)
#             tabulatedData["gamma"].append(gammaText.value)
            
#         else:
#             layerList.append((materialMenu.value,float(thicknessText.value)))
#             tabulatedData["material"].append(materialMenu.value)
#             tabulatedData["thickness"].append(thicknessText.value)
#             tabulatedData["mu"].append("/")
#             tabulatedData["gamma"].append("/")
        
#         columns = [
#                     TableColumn(field="material", title="Material"),
#                     TableColumn(field="thickness", title="thickness (nm)"),
#                     TableColumn(field="mu", title="chemical potential (cm-1)"),
#                     TableColumn(field="gamma", title="scatting rate ()")
#                   ]
        
#         source1 = ColumnDataSource(tabulatedData)
#         source.data = source1.data
#         global table
#         table.update()
    
#     #Assemble a LayeredMedia
#     def assemble_layered_media():
#         global entranceMaterial,exitMaterial
#         entranceMaterial = _get_computer_material_(entranceMaterialMenu.value)
#         exitMaterial = _get_computer_material_(exitMaterialMenu.value)
        
#         for layer in layerList:
#             materialString = layer[0]
#             if materialString=="SingleLayerGraphene":
#                 surface = SingleLayerGraphene(chemical_potential=layer[1],gamma=layer[2])
#                 computerLayerList.append(surface)
#             else:
#                 thickness = layer[1]*1e-7
#                 if materialString=='Si_80K' or materialString=='Si_150K' or materialString=='Si_300K':
#                     computerLayer = TabulatedMaterialFromFile(materialString+'.csv')
#                 else:
#                     computerLayer = _get_computer_material_(materialString)
#                 computerLayerList.append((computerLayer,thickness))
#         global LM
#         LM = LayeredMediaTM(layerArrayGUIInput = computerLayerList, entrance=entranceMaterial,exit=exitMaterial)
            
#     #Clear all inputs and stored values
#     def clear_all():
#         layerList.clear()
#         computerLayerList.clear()
#         layerMedia = None
        
#         tabulatedData["material"] = []
#         tabulatedData["thickness"] = []
#         columns = [
#             TableColumn(field="material", title="Material"),
#             TableColumn(field="thickness", title="thickness (nm)"),
#         ]
#         source1 = ColumnDataSource(tabulatedData)
#         source.data = source1.data
#         global table
#         table.update()
    
#     add.on_click(update_material_parameters)
#     assemble.on_click(assemble_layered_media)
#     clear.on_click(clear_all)
    
#     box0 = widgetbox(entranceMaterialMenu,exitMaterialMenu)
#     box01 = widgetbox(materialMenu)
#     box1 = widgetbox(muText,gammaText,css_classes = ['SingleLayerGraphene','hidden'])
#     box2 = widgetbox(thicknessText,css_classes = material_list_without_surface)
#     box3 = widgetbox(add,assemble,clear)
    
#     l1 = layout([box0,box01,box1,box2,box3],table, sizing_mode='fixed')
    
    
#     #########################################
#     ##### tab 2: Calculation parameters #####
#     ##################################################################################################################
    
#     # standard widgets
#     if len(layerList)==1 and type(computerLayerList[0])!=type(SingleLayerGraphene()):
#         quantity_list = ["permittivity",
#                          "reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
#                          "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
#                          "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
#                          "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
#                          "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
#     else:
#         quantity_list = ["reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
#                          "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
#                          "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
#                          "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
#                          "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
#     quantityMenu = Select(title="Quantity to calculate:", options=quantity_list, value="reflection_coefficient(dispersion)")
#     partMenu = Select(title="Part to calculate:", options=["real","imaginary","absolute","angle"], value="absolute")
#     polarizationMenu = Select(title="Polarization", options=['p','s'], value='p')
#     calculate_button = Button(label="Calculate!", button_type="success")
    
#     #dispersion-related input widgets
#     minFreq = TextInput(title="Min frequency: ",value="200")
#     maxFreq = TextInput(title="Max frequency: ",value="2000")
#     freqNum = TextInput(title="# of frequency points: ",value="300")
#     singleQ = TextInput(title="q value: ",value="333333")
    
#     minQ = TextInput(title="Min in-plane momentum (q): ",value="1")
#     maxQ = TextInput(title="Max q: ",value="1e6")
#     qNum = TextInput(title="# of q: ",value="300")
#     singleFreq = TextInput(title="Frequency value: ",value="1000")
    
#     minQuantityValue = TextInput(title="Min quantity value (currently only for dispersion): ",value="0")
#     maxQuantityValue = TextInput(title="Max quantity value (currently only for dispersion): ",value="1e-5")
    
#     #H field and kernel input widget
#     indexList = _get_index_list_(layerList)
#     index = Select(title="Index of interface (1 denotes the interface between entrance and first layer): ",
#                    value="1",options=indexList)
    
#     #s-SNOM signal input widgets
#     size = TextInput(title="Tip apex radius size (nm): ",value="10")
#     oscAmplitude = TextInput(title="Tip oscillation amplitude (nm): ",value="80")
#     SNOMqNum = TextInput(title="# of q points: ",value="244")
#     zNum = TextInput(title="# of z points: ",value="40")
#     normFreq = TextInput(title="Normalization frequency (cm^-1): ",value="200")
#     material_list = ["Au","SiO2_300nm","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
#     normMat = Select(title="Normalization material:", value="Au", options=material_list)
    
#     quantity_code = """
#     to_show = cb_obj.value;
#     options = cb_obj.options

#     for (i=0;i<options.length;i++) {
#         box = document.getElementsByClassName(options[i])[0];
#         if (!box.className.includes("hidden")) {box.className+=" hidden";}
#         if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
#     }
#     """
#     quantityMenu.callback = CustomJS(code=quantity_code)
    
#     def calculate_parameters():
        
#         global graph,result
#         graph = figure(plot_width=500, plot_height=400)
        
#         try:
#             if quantityMenu.value[3:]=="signal":

#                 minFreqValue = int(minFreq.value)
#                 maxFreqValue = int(maxFreq.value)
#                 freqNumValue = int(freqNum.value)
#                 freqs = np.linspace(minFreqValue,maxFreqValue,freqNumValue)

#                 a=float(size.value)*1e-7
#                 amplitude=float(oscAmplitude.value)*1e-7
#                 Nqs=int(SNOMqNum.value)
#                 Nzs=int(zNum.value)
#                 normFrequency=float(normFreq.value)
#                 matString=normMat.value
#                 normMaterial = _get_computer_material_(matString)

#                 calculated=T.LightningRodModel(freqs,rp=LM.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
#                                                     amplitude=amplitude,normalize_to=normMaterial.reflection_p,\
#                                                     normalize_at=normFrequency)
#                 result = calculated['signal_'+quantityMenu.value[1]]
                
#                 # get the part (real/imag/abs/angle) of the calculated quantities
#                 if partMenu.value=="real":
#                     result = np.real(result)
#                 elif partMenu.value=="imaginary":
#                     result = np.imag(result)
#                 elif partMenu.value=="absolute":
#                     result = abs(result)
#                 elif partMenu.value=="angle":
#                     result = np.angle(result)
#                 else:
#                     Logger.raiseException('Invalid part menu value.',exception=ValueError)
                
#                 #plotting
#                 graph.xaxis.axis_label = 'frequency (cm-1)'
#                 graph.title.text=exitMaterial.value
#                 graph.title.align="center"
#                 graph.line(freqs,result,legend=quantityMenu.value,line_color="#0093dd",line_width=2)
                
#             elif quantityMenu.value=="permittivity":
#                 minFreqValue = float(minFreq.value)
#                 maxFreqValue = float(maxFreq.value)
#                 freqNumValue = int(freqNum.value)
#                 freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
#                 eps=LM.epsilon(freq)
                
#                 # get the part (real/imag/abs/angle) of the calculated quantities
#                 if partMenu.value=="real":
#                     result = eps.real
#                 elif partMenu.value=="imaginary":
#                     result = eps.imag
#                 elif partMenu.value=="absolute":
#                     result = abs(eps.real+1j*eps.imag)
#                 elif partMenu.value=="angle":
#                     result = np.angle(eps.real+1j*eps.imag)
#                 else:
#                     Logger.raiseException('Invalid part menu value.',exception=ValueError)
                
#                 graph.xaxis.axis_label = 'frequency (cm-1)'
#                 graph.line(freq,result,line_color="#0093dd",line_width=2)
#                 graph.title.text=_get_name_of_LM(layerList,computerLayerList)
#                 graph.title.align="center"
#                 graph.yaxis.axis_label = 'Permittivity'

#             else:
#                 polarization = polarizationMenu.value
#                 T = MatrixBuilder.TransferMatrix(LM,polarization=polarization)
#                 C = Calculator.Calculator(T)

#                 # obtain appropriate dimension of freq and q
#                 if quantityMenu.value[-12:]=="(dispersion)":
#                     minFreqValue = float(minFreq.value)
#                     maxFreqValue = float(maxFreq.value)
#                     freqNumValue = int(freqNum.value)
#                     freqIn = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
#                     minQValue = float(minQ.value)
#                     maxQValue = float(maxQ.value)
#                     qNumValue = int(qNum.value)
#                     qIn = np.linspace(minQValue,maxQValue,qNumValue) 
#                     q,freq = np.meshgrid(qIn,freqIn)
#                 elif quantityMenu.value[-10:]=="(single_f)":
#                     freq = float(singleFreq.value)
#                     minQValue = float(minQ.value)
#                     maxQValue = float(maxQ.value)
#                     qNumValue = int(qNum.value)
#                     q = np.linspace(minQValue,maxQValue,qNumValue)    
#                 elif quantityMenu.value[-10:]=="(single_q)":
#                     q = float(singleQ.value)
#                     minFreqValue = float(minFreq.value)
#                     maxFreqValue = float(maxFreq.value)
#                     freqNumValue = int(freqNum.value)
#                     freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
#                 else:
#                     Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

#                 # calculate the optical quantities (based on the previous freq and q)
#                 if quantityMenu.value[:10]=="reflection":
#                     graphTitle = "r_"+polarizationMenu.value
#                     C.assemble_analytical_reflection_coefficient()
#                     result = C.get_numerical_reflection_coefficient(freq,q)
#                 elif quantityMenu.value[:12]=="transmission":
#                     graphTitle = "t_"+polarizationMenu.value
#                     C.assemble_analytical_transmission_coefficient()
#                     result = C.get_numerical_transmission_coefficient(freq,q)
#                 elif quantityMenu.value[:7]=="H_field":
#                     graphTitle = "H field"
#                     n = int(index.value)
#                     C.assemble_analytical_H_field(n, "before")
#                     result = C.get_numerical_H_field(freq, q)       
#                 elif quantityMenu.value[:7]=="Coulomb":
#                     graphTitle = "Coulomb kernel"
#                     n = int(index.value)
#                     C.assemble_analytical_kernel(n, "before")
#                     result = C.get_numerical_kernel(freq, q)
#                 else:
#                     Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

#                 # get the part (real/imag/abs/angle) of the calculated quantities
#                 if partMenu.value=="real":
#                     result = np.real(result)
#                 elif partMenu.value=="imaginary":
#                     result = np.imag(result)
#                 elif partMenu.value=="absolute":
#                     result = abs(result)
#                 elif partMenu.value=="angle":
#                     result = np.angle(result)
#                 else:
#                     Logger.raiseException('Invalid part menu value.',exception=ValueError)

#                 #create graph
#                 if type(freq)==float:
#                     xRange = q
#                     graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
#                     graph.yaxis.axis_label = graphTitle
#                     graph.xaxis.axis_label = 'q (cm$^{-1}$)'
#                     graph.title.text=_get_name_of_LM(layerList,computerLayerList)
#                     graph.title.align="center"
#                 elif type(q)==float:
#                     xRange = freq
#                     graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
#                     graph.yaxis.axis_label = graphTitle
#                     graph.xaxis.axis_label = 'Frequency (cm$^{-1}$)'
#                     graph.title.text=_get_name_of_LM(layerList,computerLayerList)
#                     graph.title.align="center"
#                 elif type(freq)==np.ndarray and type(q)==np.ndarray:
#                     graph.y_range = Range1d(freqIn.min(),freqIn.max())
#                     graph.x_range = Range1d(qIn.min(),qIn.max())
#                     graph.add_layout(Title(text=graphTitle+' dispersion', text_font_style="italic"), 'above')
#                     graph.title.align="center"
#                     graph.add_layout(Title(text=_get_name_of_LM(layerList,computerLayerList)), 'above')
#                     graph.title.align="center"
                    
#                     color_mapper = LinearColorMapper(palette="Viridis256", \
#                                                      low=float(minQuantityValue.value), high=float(maxQuantityValue.value))
#                     graph.image(image=[result], color_mapper=color_mapper,
#                                dh=[freqIn.max()], dw=[qIn.max()], x=[qIn.min()], y=[freqIn.min()])
#                     graph.xaxis.axis_label = 'q (cm-1)'
#                     graph.yaxis.axis_label = 'freq (cm-1)'
#                     color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
#                                          label_standoff=12, border_line_color=None, location=(0,0))
#                     graph.add_layout(color_bar, 'right')
#                 else:
#                     Logger.raiseException('Invalid frequency or q value.',exception=ValueError)
                
#             graph.xgrid.grid_line_color = None
#             graph.ygrid.grid_line_color = None
                
#         except ValueError:    #plot a thick red cross on panel
#             x1 = np.linspace(0, 10, 100)
#             y1 = x1
#             y2 = 10-x1
#             global x
#             x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
#             global xx
#             xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
#             graph.title.text="ERROR"
#             graph.title.align="center"
    
#     calculate_button.on_click(calculate_parameters)
    
#     menuBox = row(quantityMenu,partMenu)
#     SNOMbox = column(row(size, oscAmplitude), row(SNOMqNum, zNum), row(normFreq, normMat),
#                   css_classes = ['s0_signal','s1_signal','s2_signal','s3_signal','s4_signal','hidden'])
    
#     #Missing commas in the css_classes lists of following Boxes are intentional to ensure that widget display properly. Don't modify!
#     qRangeBox = row(minQ, maxQ, qNum,
#                   css_classes = ['reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
#                                  'reflection_coefficient(single_f)','transmission_coefficient(single_f)',
#                                  'H_field(single_f)','Coulomb_kernel(single_f)'])
    
#     freqRangeBox = row(minFreq, maxFreq,freqNum,
#                 css_classes = ['permittivity',
#                                'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
#                                'reflection_coefficient(single_q)','transmission_coefficient(single_q)',
#                                'H_field(single_q)','Coulomb_kernel(single_q)'
#                               's0_signal','s1_signal','s2_signal','s3_signal','s4_signal'])
    
#     qValueBox = column(singleFreq,
#                   css_classes = ['reflection_coefficient(single_f)','transmission_coefficient(single_f)'
#                                  'H_field(single_f)','Coulomb_kernel(single_f)','hidden'])

#     freqValueBox = column(singleQ,
#                   css_classes = ['reflection_coefficient(single_q)','transmission_coefficient(single_q)'
#                                  'H_field(single_q)','Coulomb_kernel(single_q)','hidden'])
    
#     indexBox = column(index,css_classes = ['H_field(single_f)','H_field(single_q)','H_field(dispersion)',
#                                            'Coulomb_kernel(single_f)','Coulomb_kernel(single_q)',
#                                            'Coulomb_kernel(dispersion)','hidden'])
    
#     quantityValueBox = row(minQuantityValue,maxQuantityValue)
    
#     polarizationBox = row(polarizationMenu, 
#                           css_classes=[''
#                                        'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)'
#                                        'reflection_coefficient(single_q)','transmission_coefficient(single_q)'
#                                        'reflection_coefficient(single_f)','transmission_coefficient(single_f)'])
    
#     lastBox = row(calculate_button)

#     l2 = layout(menuBox,polarizationBox,freqRangeBox,SNOMbox,qRangeBox,qValueBox,freqValueBox,indexBox,quantityValueBox,lastBox,sizing_mode='fixed')
    
#     #######################
#     ##### tab 3: Plot #####
#     ##################################################################################################################
    
#     #widgets
#     plotPanel = RadioButtonGroup(
#         labels=["Plot in 1st panel", "Plot in 2nd panel", "Plot in 3rd panel","Plot in 4th panel"], active=0)
    
#     plot_button = Button(label="Plot!", button_type="success")
    
#     # graphs
#     global s1,s2,s3,s4
    
#     s1 = figure(plot_width=600, plot_height=400)
#     s1.xgrid.grid_line_color = None
#     s1.ygrid.grid_line_color = None

#     s2 = figure(plot_width=600, plot_height=400)
#     s2.xgrid.grid_line_color = None
#     s2.ygrid.grid_line_color = None

#     s3 = figure(plot_width=600, plot_height=400)
#     s3.xgrid.grid_line_color = None
#     s3.ygrid.grid_line_color = None

#     s4 = figure(plot_width=600, plot_height=400)
#     s4.xgrid.grid_line_color = None
#     s4.ygrid.grid_line_color = None
    
#     def plot_graph():
#         #choose panel to plot
#         if plotPanel.active==0:
#             s1.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
#         elif plotPanel.active==1:
#             s2=graph
#         elif plotPanel.active==2:
#             s3=graph
#         elif plotPanel.active==3:
#             s4=graph
#         else:
#             s1=graph
    
#     plot_button.on_click(plot_graph)
    
#     l3 = layout([row(s1,s2),row(s3,s4),row(plotPanel,plot_button)],sizing_mode='fixed')
    
#     ##################################################################################################################
    
    
#     tab1 = Panel(child=l1,title="Material input")
#     tab2 = Panel(child=l2,title="Calculation parameters")
#     tab3 = Panel(child=l3,title='Plots')
#     tabs = Tabs(tabs=[tab1, tab2, tab3],width=500)
    
#     grandL = layout(header(),space(),tabs)
    
#     doc.add_root(grandL)

# show(modify_doc)

# restructure plot

In [12]:
def modify_doc(doc):
    
    #################################
    ##### tab 1: Material input #####
    ##################################################################################################################
    global layerList,computerLayerList,tabulatedData
    tabulatedData = {}
    layerList = []
    computerLayerList = []
    
    material_list = ["Au","SiO2","SingleLayerGraphene","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
    material_list_without_surface = copy.copy(material_list)
    material_list_without_surface.remove("SingleLayerGraphene")
    
    entranceMaterialMenu = Select(title="Entrance material:", value="Air", options=sorted(material_list))
    exitMaterialMenu = Select(title="Exit material:", value="Air", options=sorted(material_list))
    materialMenu = Select(title="Material", options=sorted(material_list), value="SiO2",width=200)
    
    thicknessText = TextInput(title="Thickness (nm):", value="30")
    muText = TextInput(title="Chemical potential (cm-1):", value="1000")
    gammaText = TextInput(title="Scattering rate ():", value="0.1")
    
    add = Button(label="Add", button_type="primary")
    assemble = Button(label="ASSEMBLE!", button_type="success")
    clear = Button(label="CLEAR ALL", button_type="warning")
    
    tabulatedData["material"] = []
    tabulatedData["thickness"] = []
    tabulatedData["mu"] = []
    tabulatedData["gamma"] = []
    
    source1 = ColumnDataSource(tabulatedData)
    source = source1
    columns = [
            TableColumn(field="material", title="Material"),
            TableColumn(field="thickness", title="thickness (nm)"),
            TableColumn(field="mu", title="chemical potential (cm-1)"),
            TableColumn(field="gamma", title="scatting rate ()"),
        ]
    table = DataTable(source=source, columns=columns, width=700, height=280)
    
    select_code = """
    to_show = cb_obj.value;
    options = cb_obj.options

    for (i=0;i<options.length;i++) {
        box = document.getElementsByClassName(options[i])[0];
        if (!box.className.includes("hidden")) {box.className+=" hidden";}
        if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
    }
    """
    
    materialMenu.callback = CustomJS(code=select_code)
    
    #Add a layer
    def update_material_parameters():
        
        if materialMenu.value=="SingleLayerGraphene":
            layerList.append(("SingleLayerGraphene",muText.value,gammaText.value))
            tabulatedData["material"].append("SingleLayerGraphene")
            tabulatedData["thickness"].append("/")
            tabulatedData["mu"].append(muText.value)
            tabulatedData["gamma"].append(gammaText.value)
            
        else:
            layerList.append((materialMenu.value,float(thicknessText.value)))
            tabulatedData["material"].append(materialMenu.value)
            tabulatedData["thickness"].append(thicknessText.value)
            tabulatedData["mu"].append("/")
            tabulatedData["gamma"].append("/")
        
        columns = [
                    TableColumn(field="material", title="Material"),
                    TableColumn(field="thickness", title="thickness (nm)"),
                    TableColumn(field="mu", title="chemical potential (cm-1)"),
                    TableColumn(field="gamma", title="scatting rate ()")
                  ]
        
        source1 = ColumnDataSource(tabulatedData)
        source.data = source1.data
        global table
        table.update()
    
    #Assemble a LayeredMedia
    def assemble_layered_media():
        global entranceMaterial,exitMaterial
        entranceMaterial = _get_computer_material_(entranceMaterialMenu.value)
        exitMaterial = _get_computer_material_(exitMaterialMenu.value)
        
        for layer in layerList:
            materialString = layer[0]
            if materialString=="SingleLayerGraphene":
                surface = SingleLayerGraphene(chemical_potential=float(layer[1]),gamma=float(layer[2]))
                computerLayerList.append(surface)
            else:
                thickness = layer[1]*1e-7
                if materialString=='Si_80K' or materialString=='Si_150K' or materialString=='Si_300K':
                    computerLayer = TabulatedMaterialFromFile(materialString+'.csv')
                else:
                    computerLayer = _get_computer_material_(materialString)
                computerLayerList.append((computerLayer,thickness))
        global LM
        LM = LayeredMediaTM(layerArrayGUIInput = computerLayerList, entrance=entranceMaterial,exit=exitMaterial)
            
    #Clear all inputs and stored values
    def clear_all():
        layerList.clear()
        computerLayerList.clear()
        layerMedia = None
        
        tabulatedData["material"] = []
        tabulatedData["thickness"] = []
        columns = [
            TableColumn(field="material", title="Material"),
            TableColumn(field="thickness", title="thickness (nm)"),
        ]
        source1 = ColumnDataSource(tabulatedData)
        source.data = source1.data
        global table
        table.update()
    
    add.on_click(update_material_parameters)
    assemble.on_click(assemble_layered_media)
    clear.on_click(clear_all)
    
    box0 = widgetbox(entranceMaterialMenu,exitMaterialMenu)
    box01 = widgetbox(materialMenu)
    box1 = widgetbox(muText,gammaText,css_classes = ['SingleLayerGraphene','hidden'])
    box2 = widgetbox(thicknessText,css_classes = material_list_without_surface)
    box3 = widgetbox(add,assemble,clear)
    
    l1 = layout([box0,box01,box1,box2,box3],table, sizing_mode='fixed')
    
    
    #########################################
    ##### tab 2: Calculation parameters #####
    ##################################################################################################################
    
    # graphs
    global s1,s2,s3,s4

    s1 = figure(plot_width=600, plot_height=400)
    s1.xgrid.grid_line_color = None
    s1.ygrid.grid_line_color = None

    s2 = figure(plot_width=600, plot_height=400)
    s2.xgrid.grid_line_color = None
    s2.ygrid.grid_line_color = None

    s3 = figure(plot_width=600, plot_height=400)
    s3.xgrid.grid_line_color = None
    s3.ygrid.grid_line_color = None

    s4 = figure(plot_width=600, plot_height=400)
    s4.xgrid.grid_line_color = None
    s4.ygrid.grid_line_color = None

    plotPanel = RadioButtonGroup(
        labels=["Plot in 1st panel", "Plot in 2nd panel", "Plot in 3rd panel","Plot in 4th panel"], active=0)

    # standard widgets
    if len(layerList)==1 and type(computerLayerList[0])!=type(SingleLayerGraphene()):
        quantity_list = ["permittivity",
                         "reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
                         "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
                         "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
                         "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
                         "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
    else:
        quantity_list = ["reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
                         "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
                         "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
                         "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
                         "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
    quantityMenu = Select(title="Quantity to calculate:", options=quantity_list, value="reflection_coefficient(dispersion)")
    partMenu = Select(title="Part to calculate:", options=["real","imaginary","absolute","angle"], value="absolute")
    polarizationMenu = Select(title="Polarization", options=['p','s'], value='p')
    calculate_button = Button(label="Calculate!", button_type="success")

    #dispersion-related input widgets
    minFreq = TextInput(title="Min frequency: ",value="200")
    maxFreq = TextInput(title="Max frequency: ",value="2000")
    freqNum = TextInput(title="# of frequency points: ",value="300")
    singleQ = TextInput(title="q value: ",value="333333")

    minQ = TextInput(title="Min in-plane momentum (q): ",value="1")
    maxQ = TextInput(title="Max q: ",value="1e6")
    qNum = TextInput(title="# of q: ",value="300")
    singleFreq = TextInput(title="Frequency value: ",value="1000")

    minQuantityValue = TextInput(title="Min quantity value (currently only for dispersion): ",value="0")
    maxQuantityValue = TextInput(title="Max quantity value (currently only for dispersion): ",value="1")

    #H field and kernel input widget
    indexList = _get_index_list_(layerList)
    index = Select(title="Index of interface (1 denotes the interface between entrance and first layer): ",
                   value="1",options=indexList)

    #s-SNOM signal input widgets
    size = TextInput(title="Tip apex radius size (nm): ",value="10")
    oscAmplitude = TextInput(title="Tip oscillation amplitude (nm): ",value="80")
    SNOMqNum = TextInput(title="# of q points: ",value="244")
    zNum = TextInput(title="# of z points: ",value="40")
    normFreq = TextInput(title="Normalization frequency (cm^-1): ",value="200")
    material_list = ["Au","SiO2_300nm","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
    normMat = Select(title="Normalization material:", value="Au", options=material_list)

    quantity_code = """
    to_show = cb_obj.value;
    options = cb_obj.options

    for (i=0;i<options.length;i++) {
        box = document.getElementsByClassName(options[i])[0];
        if (!box.className.includes("hidden")) {box.className+=" hidden";}
        if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
    }
    """
    quantityMenu.callback = CustomJS(code=quantity_code)
    
    global result,graph
    def calculate_parameters():
        
        if plotPanel.active==0:
            graph=s1
        elif plotPanel.active==1:
            graph=s2
        elif plotPanel.active==2:
            graph=s3
        elif plotPanel.active==3:
            plot_graph=s4
        else:
            graph=s1

        try:
            if quantityMenu.value[3:]=="signal":

                minFreqValue = int(minFreq.value)
                maxFreqValue = int(maxFreq.value)
                freqNumValue = int(freqNum.value)
                freqs = np.linspace(minFreqValue,maxFreqValue,freqNumValue)

                a=float(size.value)*1e-7
                amplitude=float(oscAmplitude.value)*1e-7
                Nqs=int(SNOMqNum.value)
                Nzs=int(zNum.value)
                normFrequency=float(normFreq.value)
                matString=normMat.value
                normMaterial = _get_computer_material_(matString)

                calculated=T.LightningRodModel(freqs,rp=LM.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                                    amplitude=amplitude,normalize_to=normMaterial.reflection_p,\
                                                    normalize_at=normFrequency)
                result = calculated['signal_'+quantityMenu.value[1]]

                # get the part (real/imag/abs/angle) of the calculated quantities
                if partMenu.value=="real":
                    result = np.real(result)
                elif partMenu.value=="imaginary":
                    result = np.imag(result)
                elif partMenu.value=="absolute":
                    result = abs(result)
                elif partMenu.value=="angle":
                    result = np.angle(result)
                else:
                    Logger.raiseException('Invalid part menu value.',exception=ValueError)

                #plotting
                graph.xaxis.axis_label = 'frequency (cm-1)'
                graph.title.text=exitMaterial.value
                graph.title.align="center"
                graph.line(freqs,result,legend=quantityMenu.value,line_color="#0093dd",line_width=2)

            elif quantityMenu.value=="permittivity":
                minFreqValue = float(minFreq.value)
                maxFreqValue = float(maxFreq.value)
                freqNumValue = int(freqNum.value)
                freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                eps=LM.epsilon(freq)

                # get the part (real/imag/abs/angle) of the calculated quantities
                if partMenu.value=="real":
                    result = eps.real
                elif partMenu.value=="imaginary":
                    result = eps.imag
                elif partMenu.value=="absolute":
                    result = abs(eps.real+1j*eps.imag)
                elif partMenu.value=="angle":
                    result = np.angle(eps.real+1j*eps.imag)
                else:
                    Logger.raiseException('Invalid part menu value.',exception=ValueError)

                graph.xaxis.axis_label = 'frequency (cm-1)'
                graph.line(freq,result,line_color="#0093dd",line_width=2)
                graph.title.text=_get_name_of_LM(layerList,computerLayerList)
                graph.title.align="center"
                graph.yaxis.axis_label = 'Permittivity'

            else:
                polarization = polarizationMenu.value
                T = MatrixBuilder.TransferMatrix(LM,polarization=polarization)
                C = Calculator.Calculator(T)

                # obtain appropriate dimension of freq and q
                if quantityMenu.value[-12:]=="(dispersion)":
                    minFreqValue = float(minFreq.value)
                    maxFreqValue = float(maxFreq.value)
                    freqNumValue = int(freqNum.value)
                    freqIn = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                    minQValue = float(minQ.value)
                    maxQValue = float(maxQ.value)
                    qNumValue = int(qNum.value)
                    qIn = np.linspace(minQValue,maxQValue,qNumValue) 
                    q,freq = np.meshgrid(qIn,freqIn)
                elif quantityMenu.value[-10:]=="(single_f)":
                    freq = float(singleFreq.value)
                    minQValue = float(minQ.value)
                    maxQValue = float(maxQ.value)
                    qNumValue = int(qNum.value)
                    q = np.linspace(minQValue,maxQValue,qNumValue)    
                elif quantityMenu.value[-10:]=="(single_q)":
                    q = float(singleQ.value)
                    minFreqValue = float(minFreq.value)
                    maxFreqValue = float(maxFreq.value)
                    freqNumValue = int(freqNum.value)
                    freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                else:
                    Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

                # calculate the optical quantities (based on the previous freq and q)
                if quantityMenu.value[:10]=="reflection":
                    graphTitle = "r_"+polarizationMenu.value
                    C.assemble_analytical_reflection_coefficient()
                    result = C.get_numerical_reflection_coefficient(freq,q)
                elif quantityMenu.value[:12]=="transmission":
                    graphTitle = "t_"+polarizationMenu.value
                    C.assemble_analytical_transmission_coefficient()
                    result = C.get_numerical_transmission_coefficient(freq,q)
                elif quantityMenu.value[:7]=="H_field":
                    graphTitle = "H field"
                    n = int(index.value)
                    C.assemble_analytical_H_field(n, "before")
                    result = C.get_numerical_H_field(freq, q)       
                elif quantityMenu.value[:7]=="Coulomb":
                    graphTitle = "Coulomb kernel"
                    n = int(index.value)
                    C.assemble_analytical_kernel(n, "before")
                    result = C.get_numerical_kernel(freq, q)
                else:
                    Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

                # get the part (real/imag/abs/angle) of the calculated quantities
                if partMenu.value=="real":
                    result = np.real(result)
                elif partMenu.value=="imaginary":
                    result = np.imag(result)
                elif partMenu.value=="absolute":
                    result = abs(result)
                elif partMenu.value=="angle":
                    result = np.angle(result)
                else:
                    Logger.raiseException('Invalid part menu value.',exception=ValueError)

                #create graph
                if type(freq)==float:
                    xRange = q
                    graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
                    graph.yaxis.axis_label = graphTitle
                    graph.xaxis.axis_label = 'q (cm$^{-1}$)'
                    graph.title.text=_get_name_of_LM(layerList,computerLayerList)
                    graph.title.align="center"
                elif type(q)==float:
                    xRange = freq
                    graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
                    graph.yaxis.axis_label = graphTitle
                    graph.xaxis.axis_label = 'Frequency (cm$^{-1}$)'
                    graph.title.text=_get_name_of_LM(layerList,computerLayerList)
                    graph.title.align="center"
                elif type(freq)==np.ndarray and type(q)==np.ndarray:
        #             graph = figure(toolbar_location=None)
                    graph.y_range = Range1d(freqIn.min(),freqIn.max())
                    graph.x_range = Range1d(qIn.min(),qIn.max())
                    graph.add_layout(Title(text=graphTitle+' dispersion', text_font_style="italic"), 'above')
                    graph.title.align="center"
                    graph.add_layout(Title(text=_get_name_of_LM(layerList,computerLayerList)), 'above')
                    graph.title.align="center"

                    color_mapper = LinearColorMapper(palette="Viridis256", \
                                                     low=float(minQuantityValue.value), high=float(maxQuantityValue.value))
                    graph.image(image=[result], color_mapper=color_mapper,
                               dh=[freqIn.max()], dw=[qIn.max()], x=[qIn.min()], y=[freqIn.min()])
                    graph.xaxis.axis_label = 'q (cm-1)'
                    graph.yaxis.axis_label = 'freq (cm-1)'
                    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                                         label_standoff=12, border_line_color=None, location=(0,0))
                    graph.add_layout(color_bar, 'right')
                else:
                    Logger.raiseException('Invalid frequency or q value.',exception=ValueError)

            graph.xgrid.grid_line_color = None
            graph.ygrid.grid_line_color = None

        except ValueError:    #plot a thick red cross on panel
            x1 = np.linspace(0, 10, 100)
            y1 = x1
            y2 = 10-x1
            global x
            x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
            global xx
            xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
            graph.title.text="ERROR"
            graph.title.align="center"
    
    calculate_button.on_click(calculate_parameters)
    
    menuBox = row(quantityMenu,partMenu)
    SNOMbox = column(row(size, oscAmplitude), row(SNOMqNum, zNum), row(normFreq, normMat),
                  css_classes = ['s0_signal','s1_signal','s2_signal','s3_signal','s4_signal','hidden'])
    
    #Missing commas in the css_classes lists of following Boxes are intentional to ensure that widget display properly. Don't modify!
    qRangeBox = row(minQ, maxQ, qNum,
                  css_classes = ['reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
                                 'reflection_coefficient(single_f)','transmission_coefficient(single_f)',
                                 'H_field(single_f)','Coulomb_kernel(single_f)'])
    
    freqRangeBox = row(minFreq, maxFreq,freqNum,
                css_classes = ['permittivity',
                               'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
                               'reflection_coefficient(single_q)','transmission_coefficient(single_q)',
                               'H_field(single_q)','Coulomb_kernel(single_q)'
                              's0_signal','s1_signal','s2_signal','s3_signal','s4_signal'])
    
    qValueBox = column(singleFreq,
                  css_classes = ['reflection_coefficient(single_f)','transmission_coefficient(single_f)'
                                 'H_field(single_f)','Coulomb_kernel(single_f)','hidden'])

    freqValueBox = column(singleQ,
                  css_classes = ['reflection_coefficient(single_q)','transmission_coefficient(single_q)'
                                 'H_field(single_q)','Coulomb_kernel(single_q)','hidden'])
    
    indexBox = column(index,css_classes = ['H_field(single_f)','H_field(single_q)','H_field(dispersion)',
                                           'Coulomb_kernel(single_f)','Coulomb_kernel(single_q)',
                                           'Coulomb_kernel(dispersion)','hidden'])
    
    quantityValueBox = row(minQuantityValue,maxQuantityValue)
    
    polarizationBox = row(polarizationMenu, 
                          css_classes=[''
                                       'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)'
                                       'reflection_coefficient(single_q)','transmission_coefficient(single_q)'
                                       'reflection_coefficient(single_f)','transmission_coefficient(single_f)'])
    
    lastBox = row(calculate_button,plotPanel)

    l2 = layout(menuBox,polarizationBox,freqRangeBox,SNOMbox,qRangeBox,qValueBox,freqValueBox,indexBox,quantityValueBox,lastBox,sizing_mode='fixed')
    
    #######################
    ##### tab 3: Plot #####
    ##################################################################################################################
    
    #s1-s4 is instantiated in tab 2 for processing
    
    l3 = layout([row(s1,s2),row(s3,s4)],sizing_mode='fixed')
    
    ##################################################################################################################
    
    
    tab1 = Panel(child=l1,title="Material input")
    tab2 = Panel(child=l2,title="Calculation parameters")
    tab3 = Panel(child=l3,title='Plots')
    tabs = Tabs(tabs=[tab1, tab2, tab3],width=500)
    
    grandL = layout(header(),space(),tabs)
    
    doc.add_root(grandL)

show(modify_doc)